In [3]:
# fuse_to_ttl.py
from pathlib import Path
import pandas as pd
from rdflib import Graph, Namespace, URIRef, Literal, RDF
BRICK = Namespace("https://brickschema.org/schema/Brick#")
SOSA  = Namespace("http://www.w3.org/ns/sosa/")
QUDT  = Namespace("http://qudt.org/schema/qudt/")
UNIT  = Namespace("http://qudt.org/vocab/unit/")
EX    = Namespace("http://example.org/training#")

BASE = Path.home()/ "DTE" / "jne_project"
BIM  = BASE/"Training_room_en.ttl"                      # le nouveau fichier
RAW  = BASE/"raw"
OUT  = BASE/"graph"; OUT.mkdir(parents=True, exist_ok=True)
OUT_TTL = OUT/"edtb_graph.ttl"

# CSV attendus
WX  = RAW/"weather/2025-03/weather.csv"
SEN = RAW/"sensors/2025-03/zone_101_sensors.csv"
BMS = RAW/"bms/2025-03/bms.csv"
OCC = RAW/"occupancy/2025-03/occupancy.csv"

# mappages colonne CSV → URI de capteur dans le BIM
MAP = {
  # sensors.csv
  "temp_int_c":       EX.T_Int_Sensor,
  "rh_int_pct":       EX.RH_Int_Sensor,
  "co2_ppm":          EX.CO2_Sensor,
  "pir_bin":          EX.PIR_Sensor,
  "power_total_kw":   EX.Power_Total_Sensor,
  # bms.csv
  "T_set":            EX.T_Setpoint,
  "hvac_state":       EX.HVAC_Status,
  # weather.csv
  "weather_temp_c":   EX.WX_Temp,
  "weather_rh_pct":   EX.WX_RH,
  "weather_wind_ms":  EX.WX_Wind,
  "weather_ghi_wm2":  EX.WX_GHI,
  # occupancy.csv
  "presence":         EX.Occupancy_Level,
}

UNITS = {
  "temp_int_c":"DegreeCelsius","T_set":"DegreeCelsius","weather_temp_c":"DegreeCelsius",
  "rh_int_pct":"Percent","weather_rh_pct":"Percent",
  "co2_ppm":"PPM","pir_bin":"One","hvac_state":"One","presence":"One",
  "power_total_kw":"KiloW","weather_wind_ms":"MeterPerSecond","weather_ghi_wm2":"WPerSquareMeter",
}

def read_utc(p):
    df = pd.read_csv(p, parse_dates=["ts"])
    if df["ts"].dt.tz is None: df["ts"] = df["ts"].dt.tz_localize("UTC")
    else: df["ts"] = df["ts"].dt.tz_convert("UTC")
    return df

def add_obs(g, sid:URIRef, ts, val, unit_str:str):
    if pd.isna(val): return
    oid = URIRef(str(EX) + f"obs_{abs(hash((sid,ts)))}")
    g.add((oid, RDF.type, SOSA.Observation))
    g.add((oid, SOSA.madeBySensor, sid))
    g.add((oid, SOSA.resultTime, Literal(ts.isoformat())))
    try: lit = Literal(float(val))
    except: lit = Literal(str(val))
    g.add((oid, SOSA.hasSimpleResult, lit))
    if unit_str: g.add((oid, QUDT.unit, getattr(UNIT, unit_str)))

def main():
    g = Graph(); g.bind("brick",BRICK); g.bind("sosa",SOSA); g.bind("qudt",QUDT); g.bind("unit",UNIT); g.bind("ex",EX)
    g.parse(BIM, format="turtle")  # BIM

    # weather
    wx = read_utc(WX);   wx = wx.rename(columns={"T_ext":"weather_temp_c","RH":"weather_rh_pct","wind":"weather_wind_ms","GHI":"weather_ghi_wm2"})
    for _,r in wx.iterrows():
        for c in [k for k in MAP if k.startswith("weather_")]:
            if c in r: add_obs(g, MAP[c], r["ts"], r[c], UNITS[c])

    # sensors
    sen = read_utc(SEN)
    for _,r in sen.iterrows():
        for c in ["temp_int_c","rh_int_pct","co2_ppm","pir_bin","power_total_kw"]:
            if c in r: add_obs(g, MAP[c], r["ts"], r[c], UNITS[c])

    # bms
    bms = read_utc(BMS)
    for _,r in bms.iterrows():
        for c in ["T_set","hvac_state","P_total"]:
            if c in r and c in MAP: add_obs(g, MAP[c], r["ts"], r[c], UNITS.get(c,""))

    # occupancy
    occ = read_utc(OCC)
    for _,r in occ.iterrows():
        for c in ["presence"]:
            if c in r: add_obs(g, MAP[c], r["ts"], r[c], UNITS[c])
        if "level" in r:
            add_obs(g, EX.Occupancy_Level, r["ts"], str(r["level"]), "One")

    g.serialize(OUT_TTL, format="turtle")
    print("OK →", OUT_TTL)

if __name__ == "__main__":
    main()


OK → /home/amina/DTE/jne_project/graph/edtb_graph.ttl
